Using OpenAI stable baselines to train a stock trading agent which is only compatible with up to tensorflow version 1.15.4.

In [8]:
!pip install stable-baselines
!pip install tensorflow==1.15.4

ERROR: Could not find a version that satisfies the requirement tensorflow==1.15.4 (from versions: 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.2.1, 2.2.2, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0, 2.3.1, 2.3.2, 2.4.0rc0, 2.4.0rc1, 2.4.0rc2, 2.4.0rc3, 2.4.0rc4, 2.4.0, 2.4.1)
ERROR: No matching distribution found for tensorflow==1.15.4


This jupyter notebook was originally run on Google Colab. If not being run on Google Colab, skip the following drive mounting step

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/MyDrive/Colab\ Notebooks

In [7]:
from indicators import get_indicators, sma, macd, rsi, cci
from environment import StockEnvironment

import csv
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn.utils
import torch.optim as optim
import numpy as np

import gym
from gym import spaces

ModuleNotFoundError: No module named 'tensorflow'

Reading the stock data from a csv file that was originally downloaded from kaggle. The data being read stores one set of values for each day the market is open. It contains information about the date, high, low, open, close, and adjusted close for the day.

In [2]:
data_dict = {'Date': [], 'Open': [], 'Close': [], 'High': [], 'Low': [], 'Adj Close': []}

with open('./data/stocks/AAPL.csv', newline = '') as csvfile:
  reader = csv.DictReader(csvfile)
  for row in reader:
    data_dict['Date'].append(row['Date'])
    data_dict['Open'].append(float(row['Open']))
    data_dict['Close'].append(float(row['Close']))
    data_dict['High'].append(float(row['High']))
    data_dict['Low'].append(float(row['Low']))
    data_dict['Adj Close'].append(float(row['Adj Close']))

indicators = get_indicators(data_dict)

Training the model and saving it

In [ ]:
from stable_baselines import DQN

env = StockEnvironment(data_dict, indicators, 1200)
model = DQN('MlpPolicy', env, learning_rate=5e-4, prioritized_replay=True, verbose=1)
trained_model = model.learn(300000, log_interval=5)

In [ ]:
model.save("./trained models/DQN AAPL Model")

In [6]:
from stable_baselines import DQN
model = DQN.load("./trained models/DQN AAPL Model")

ModuleNotFoundError: No module named 'tensorflow'

Computing some relevant statistics for the trading agent and printing that information. Create a plot that compares stock price to portfolio value where the starting stock price has be adjusted to be equal to the portfolio value in order to allow for a direct comparison.

In [ ]:
obs = env.reset()
action_list = [0] * 9
portfolio_value = []
day = 201

for i in range(5000):
  action, _states = model.predict(obs)
  action_list[action] += 1
  obs, rewards, done, info = env.step(action)
  portfolio_value.append(obs[0] + obs[1]*data_dict['Close'][day])
  day += 1

env.render()
print(action_list)

other_prices = [num * 367 for num in data_dict['Close']]
plt.plot(portfolio_value, label='portfolio value')
plt.plot(other_prices[200:5200], label='stock price')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()